To run this Application write this in your terminal:

```py -3.10 -m venv myenv```

```.\myenv\Scripts\Activate```

select the kernal to be myenv (Python 3.10.0)

```py --version``` insure this version is 3.10.0


In [111]:
!py --version

Python 3.10.0


- **this cell could take some time to run**

In [1]:
import subprocess
import importlib
import os
import json
from pathlib import Path
import logging
from concurrent.futures import ThreadPoolExecutor 

# Define the libraries and their pip install commands
libraries = {
    "tensorflow": "pip install tensorflow",
    "tensorflow-hub": "pip install tensorflow-hub",
    "numpy": "pip install numpy",
    "matplotlib": "pip install matplotlib",
    "pandas": "pip install pandas",
    "cv2": "pip install opencv-python", 
    "keras": "pip install keras",
    "pillow": "pip install pillow",
}

for library, command in libraries.items():
    try:
        importlib.import_module(library)
        print(f"{library} is already installed.")
    except ImportError:
        print(f"{library} not found. Installing...")
        try:
            subprocess.run(command.split(), check=True)
            print(f"{library} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {library}. Error: {e}")


import tensorflow as tf
import tensorflow_hub as hub
from keras import models, layers, applications, utils, preprocessing, callbacks, optimizers
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

tensorflow is already installed.
tensorflow-hub not found. Installing...
tensorflow-hub installed successfully.
numpy is already installed.
matplotlib is already installed.
pandas is already installed.
cv2 is already installed.
keras is already installed.
pillow not found. Installing...
pillow installed successfully.



In [2]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True) # Log device placement (on which device the operation is executed)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU 0


Num GPUs Available:  0


In [4]:
def video_to_frames(video_path, output_folder, frame_rate=1):
    """
    Extracts frames from a video at a specified frame rate.

    Args:
        video_path (str): Path to the video file.
        output_folder (str): Folder to save the extracted frames.
        frame_rate (int): Number of frames to save per second of video.

    Returns:
        None
    """
    
    if frame_rate <= 0:
        logging.error("Error: Frame rate must be a positive integer.")
        return

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        logging.error(f"Error: Could not open video file '{video_path}'.")
        return

    # Get the video's frame rate and total number of frames
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    if video_fps <= 0:
        logging.error("Error: Invalid video FPS. Please check the video file.")
        video.release()
        return

    if frame_rate > video_fps:
        logging.warning(f"Warning: Frame rate ({frame_rate}) exceeds video FPS ({video_fps}). Adjusting to match video FPS.")
        frame_rate = video_fps

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    logging.info(f"Video FPS: {video_fps}, Total Frames: {frame_count}")

    # Calculate the frame interval for the desired frame rate
    frame_interval = video_fps // frame_rate

    # Extract and save frames
    frame_idx = 0
    saved_frame_idx = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break  # End of video

        # Save the frame at the specified interval
        if frame_idx % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_idx:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_idx += 1

        frame_idx += 1

    # Release the video capture object
    video.release()

    if saved_frame_idx == 0:
        logging.warning("No frames were saved. Adjust the frame rate and try again.")
    else:
        logging.info(f"{saved_frame_idx} frames saved to '{output_folder}'.")
        
    return saved_frame_idx

In [5]:
target_extraction_folders = [
    "workspace/good_pose",
    "workspace/bad_pose",
    "workspace/cant_determine"
]


def process_session(session_path, label, output_dir, frame_rate=1):
    """
    Processes a single session folder, extracting frames and mapping to sensor data.
    """
    video_path, json_path = None, None

    for file_name in os.listdir(session_path):
        file_path = os.path.join(session_path, file_name)
        lower = file_name.lower()
        if lower.endswith('.mp4'):
            video_path = file_path
        elif lower.endswith('.json'):
            json_path = file_path

    if not video_path or not os.path.exists(video_path):
        logging.warning(f"No valid video found in: {session_path}")
        return
    if not json_path or not os.path.exists(json_path):
        logging.warning(f"No valid JSON file found in: {session_path}")
        return

    with open(json_path, 'r') as f:
        sensor_data = json.load(f)

    gyro_list = sensor_data.get("gyroscopeData", [])
    accel_list = sensor_data.get("accelerometerData", [])

    session_name = Path(session_path).name
    session_output = os.path.join(output_dir, label, session_name)
    os.makedirs(session_output, exist_ok=True)

    saved_frames = video_to_frames(video_path, session_output, frame_rate)
    if saved_frames == 0:
        logging.warning("No frames were extracted; skipping sensor mapping.")
        return

    if not gyro_list or not accel_list:
        logging.warning("Missing sensor data in JSON; skipping sensor mapping.")
        return

    frame_files = sorted([f for f in os.listdir(session_output) if f.lower().endswith('.jpg')])
    csv_path = os.path.join(session_output, "frame_sensor_mapping.csv")

    with open(csv_path, 'w') as csv_file:
        csv_file.write("frame_file,frame_idx,sensor_idx,gyro_x,gyro_y,gyro_z,accel_x,accel_y,accel_z\n")
        M = saved_frames
        N = len(gyro_list)
        L = len(accel_list)

        for i, frame_file in enumerate(frame_files):
            gyro_idx = int((i / M) * N)
            accel_idx = int((i / M) * L)

            gyro_data = gyro_list[gyro_idx]
            accel_data = accel_list[accel_idx]

            csv_file.write(
                f"{frame_file},{i},{gyro_idx},"
                f"{gyro_data.get('x', 0)},{gyro_data.get('y', 0)},{gyro_data.get('z', 0)},"
                f"{accel_data.get('x', 0)},{accel_data.get('y', 0)},{accel_data.get('z', 0)}\n"
            )

            # old_path = os.path.join(session_output, frame_file)
            # base, ext = os.path.splitext(frame_file)
            # new_filename = f"{base}_sensor_{gyro_idx}{ext}"
            # new_path = os.path.join(session_output, new_filename)
            # os.rename(old_path, new_path)

    logging.info(f"Frame-to-sensor mapping complete for: {session_path}")

In [8]:
output_dir = "extracted_frames"
# frame_rate = 30
frame_rate = 10 # for demonstration purposes

def process_label(label):
        label_path = Path(label)
        if not label_path.exists():
            logging.warning(f"Label folder not found: {label_path}")
            return

        for session_name in os.listdir(label_path):
            session_path = os.path.join(label_path, session_name)
            if not os.path.isdir(session_path):
                continue

            try:
                process_session(session_path, label, output_dir, frame_rate)
            except Exception as e:
                logging.error(f"Error processing session '{session_path}': {e}")

with ThreadPoolExecutor() as executor:
    executor.map(process_label, target_extraction_folders)

2025-01-14 14:17:22,744 - INFO - Video FPS: 29, Total Frames: 751


2025-01-14 14:17:22,803 - INFO - Video FPS: 30, Total Frames: 525
2025-01-14 14:17:22,811 - INFO - Video FPS: 29, Total Frames: 1217
2025-01-14 14:17:28,676 - INFO - 175 frames saved to 'extracted_frames\workspace/bad_pose\10'.
2025-01-14 14:17:28,677 - WARNING - Missing sensor data in JSON; skipping sensor mapping.
2025-01-14 14:17:28,689 - INFO - Video FPS: 29, Total Frames: 311
2025-01-14 14:17:32,889 - INFO - 156 frames saved to 'extracted_frames\workspace/bad_pose\12'.
2025-01-14 14:17:32,898 - INFO - Frame-to-sensor mapping complete for: workspace\bad_pose\12
2025-01-14 14:17:32,910 - INFO - Video FPS: 29, Total Frames: 1018
2025-01-14 14:17:33,309 - INFO - 376 frames saved to 'extracted_frames\workspace/good_pose\1'.
2025-01-14 14:17:33,314 - INFO - Frame-to-sensor mapping complete for: workspace\good_pose\1
2025-01-14 14:17:33,335 - INFO - Video FPS: 29, Total Frames: 667
2025-01-14 14:17:39,535 - INFO - 609 frames saved to 'extracted_frames\workspace/cant_determine\14'.
2025-0

In [117]:
import pandas as pd
mapped_df = pd.read_csv("extracted_frames/workspace/good_pose/1/frame_sensor_mapping.csv")
mapped_df.head()

,frame_file,frame_idx,sensor_idx,gyro_x,gyro_y,gyro_z,accel_x,accel_y,accel_z
0,frame_0000.jpg,0,0,-0.384450,0.453337,0.583412,-0.05100,7.07805,-7.16805
1,frame_0001.jpg,1,99,-0.007425,-0.029288,0.002887,-0.31200,7.41105,-6.30105
2,frame_0002.jpg,2,198,-0.036162,-0.098587,-0.021588,-0.23700,7.63905,-5.96805
3,frame_0003.jpg,3,297,0.049087,-0.113437,-0.003437,-0.54900,7.70595,-5.63295
4,frame_0004.jpg,4,396,-0.060638,0.000550,0.007150,-0.49095,7.80195,-5.57295


In [3]:
IMAGE_SIZE = (224, 224)
DATA_DIR = "extracted_frames/workspace"
DATA_COLS = ["gyro_x", "gyro_y", "gyro_z", "accel_x", "accel_y", "accel_z"]

In [9]:
# from keras import layers
def augment_image(img):
    """
    Apply data augmentation to a single image.

    Args:
        img (np.ndarray): Input image as a NumPy array.

    Returns:
        np.ndarray: Augmented image as a NumPy array.
    """
    img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
    img_tensor = tf.image.random_flip_left_right(img_tensor)  # Random horizontal flip
    img_tensor = tf.image.random_brightness(
        img_tensor, max_delta=0.3
    )  # Random brightness
    img_tensor = tf.image.random_contrast(
        img_tensor, lower=0.8, upper=1.2
    )  # Random contrast
    img_tensor = tf.image.resize_with_crop_or_pad(
        img_tensor, IMAGE_SIZE[0] + 10, IMAGE_SIZE[1] + 10
    )  # Add padding
    img_tensor = tf.image.random_crop(
        img_tensor, size=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
    )  # Random crop
    img_tensor = tf.image.random_saturation(
        img_tensor, lower=0.5, upper=1.5
    )  # playing with the saturation


    img_tensor = tf.clip_by_value(img_tensor, 0.0, 255.0)  # Clip values
    return img_tensor.numpy()

In [ ]:
[[0, 0, 1], [1, 0, 0]]

In [1]:
def load_data(data_dir, min_frames_per_session=1, augmentations_per_image=3):
    """
    Loads images, sensor data, and labels from the dataset directory, with data augmentation.

    Args:
        data_dir (str): Path to the dataset directory.
        min_frames_per_session (int): Minimum number of frames required to process a session.
        augmentations_per_image (int): Number of augmented images to generate per original image.

    Returns:
        Tuple: (images, sensors, labels)
    """
    images = []
    sensors = []
    labels = []
    
    # "cant_determien",
    for label_folder in ["good_pose", "bad_pose"]:
        label_path = os.path.join(data_dir, label_folder)
        if not os.path.exists(label_path):
            logging.warning(f"Label folder not found: {label_path}")
            continue

        # Loop through each session folder
        session_folders = [f for f in os.listdir(label_path) if os.path.isdir(os.path.join(label_path, f))]
        for session in session_folders:
            session_path = os.path.join(label_path, session)

            # Find the sensor mapping CSV
            csv_file = os.path.join(session_path, "frame_sensor_mapping.csv")
            if not os.path.exists(csv_file):
                logging.warning(f"Sensor mapping CSV not found: {csv_file}")
                continue

            # Load the sensor data from CSV
            df = pd.read_csv(csv_file)
            session_images = []
            session_sensors = []
            session_labels = []

            # Get all available frame filenames in the session directory
            available_frames = {os.path.basename(f) for f in os.listdir(session_path) if f.endswith('.jpg')}
            logging.debug(f"Available frames in {session_path}: {available_frames}")

            for _, row in df.iterrows():
                frame_file_name = row["frame_file"]
                if frame_file_name not in available_frames:
                    logging.warning(f"Frame file not found: {frame_file_name}")
                    continue

                frame_file_path = os.path.join(session_path, frame_file_name)
                try:
                    # Load and preprocess image
                    img = preprocessing.image.load_img(frame_file_path, target_size=IMAGE_SIZE)
                    img = preprocessing.image.img_to_array(img) / 255.0  # Normalize to [0, 1]

                    # Apply augmentations
                    # augmented_imgs = [img] + [augment_image(img) for _ in range(augmentations_per_image)]
                    augmented_imgs = [img]
                    # Load sensor data
                    sensor_features = np.array([
                        row["gyro_x"], row["gyro_y"], row["gyro_z"],
                        row["accel_x"], row["accel_y"], row["accel_z"]
                    ])

                    # Append data
                    for augmented_img in augmented_imgs:
                        session_images.append(augmented_img)
                        session_sensors.append(sensor_features)
                        session_labels.append(1 if label_folder == "good_pose" else 0)
                except Exception as e:
                    logging.error(f"Error processing frame {frame_file_path}: {e}")
                    continue

            # Check if session has enough frames
            if len(session_images) < min_frames_per_session:
                logging.warning(f"Session skipped due to insufficient frames: {session_path}")
                continue

            # Add session data to global lists
            images.extend(session_images)
            sensors.extend(session_sensors)
            labels.extend(session_labels)

    images = np.array(images, dtype=np.float32)
    sensors = np.array(sensors, dtype=np.float32)
    labels = utils.to_categorical(np.array(labels, dtype=np.int32), num_classes=2)

    logging.info(f"Loaded {len(labels)} samples (including augmented) from {data_dir}.")
    return images, sensors, labels

In [187]:
img = preprocessing.image.load_img('extracted_frames/workspace/bad_pose/10/frame_0000.jpg', target_size=IMAGE_SIZE)
# print(img)
img_array = preprocessing.image.img_to_array(img)
# print(img_array)
augmented_imgs = [img_array] + [augment_image(img) for _ in range(10)]

# store teh augmented images in a directory
augmented_dir = "augmented_images"
os.makedirs(augmented_dir, exist_ok=True)
for i, img in enumerate(augmented_imgs):
    img_path = os.path.join(augmented_dir, f"augmented_{i}.jpg")
    img_pil = preprocessing.image.array_to_img(img)
    img_pil.save(img_path)

In [11]:
def split_data(images, sensors, labels, validation_split=0.2, test_split=0.1):
    """
    Splits the dataset into training, testing, and validation sets.
    
    Args:
        images (np.ndarray): Array of image data.
        sensors (np.ndarray): Array of sensor data.
        labels (np.ndarray): Array of labels.
        validation_split (float): Proportion of the data to use for validation.
        test_split (float): Proportion of the data to use for testing.
        
    Returns:
        Tuple: (X_img_train, X_img_val, X_img_test, 
                X_sensor_train, X_sensor_val, X_sensor_test, 
                y_train, y_val, y_test)
    """
    dataset_size = len(images)
    test_size = int(dataset_size * test_split)
    val_size = int(dataset_size * validation_split)
    train_size = dataset_size - test_size - val_size

    # Shuffle indices
    indices = np.arange(dataset_size)
    np.random.shuffle(indices)

    # Split indices
    test_indices = indices[:test_size]
    val_indices = indices[test_size:test_size + val_size]
    train_indices = indices[test_size + val_size:]

    # Slice data
    X_img_train = images[train_indices]
    X_img_val = images[val_indices]
    X_img_test = images[test_indices]
    X_sensor_train = sensors[train_indices]
    X_sensor_val = sensors[val_indices]
    X_sensor_test = sensors[test_indices]
    y_train = labels[train_indices]
    y_val = labels[val_indices]
    y_test = labels[test_indices]

    return (X_img_train, X_img_val, X_img_test, 
            X_sensor_train, X_sensor_val, X_sensor_test, 
            y_train, y_val, y_test)


In [ ]:
images, sensors, labels = load_data(DATA_DIR)
# Split data using custom splitting logic

print(
    f"Loaded data: Images - {images.shape}, Sensors - {sensors.shape}, Labels - {labels.shape}"
)

In [16]:
X_img_train, X_img_val, X_img_test, X_sensor_train, X_sensor_val, X_sensor_test, y_train, y_val, y_test = split_data(
    images, sensors, labels, validation_split=0.2, test_split=0.1
)

# Check shapes
print("Image training data shape:", X_img_train.shape)
print("Sensor training data shape:", X_sensor_train.shape)
print("Label training data shape:", y_train.shape)
print("Image validation data shape:", X_img_val.shape)
print("Sensor validation data shape:", X_sensor_val.shape)
print("Label validation data shape:", y_val.shape)
print("Image testing data shape:", X_img_test.shape)
print("Sensor testing data shape:", X_sensor_test.shape)
print("Label testing data shape:", y_test.shape)

Image training data shape: (11142, 224, 224, 3)
Sensor training data shape: (11142, 6)
Label training data shape: (11142, 2)
Image validation data shape: (3183, 224, 224, 3)
Sensor validation data shape: (3183, 6)
Label validation data shape: (3183, 2)
Image testing data shape: (1591, 224, 224, 3)
Sensor testing data shape: (1591, 6)
Label testing data shape: (1591, 2)


In [203]:
# X_img_train
# X_sensor_train
# y_train

# store the data of the training array in a file for showing the data
np.save("X_img_train.npy", 
        # small sample of the data
        X_img_train[:10]
        )
np.save("X_sensor_train.npy", X_sensor_train[:10])
np.save("y_train.npy", y_train[:10])


In [ ]:
# Image input
image_input = layers.Input(shape=(224, 224, 3))

base_model = applications.MobileNetV2(
    weights="imagenet", include_top=False, input_tensor=image_input
)

image_features = layers.GlobalAveragePooling2D()(base_model.output)

# Sensor input
sensor_input = layers.Input(shape=(6,))  # 3 gyroscope + 3 accelerometer features
sensor_features = layers.Dense(
    64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)
)(sensor_input)

# Combine features
combined = layers.concatenate([image_features, sensor_features])
combined = layers.Dropout(0.5)(combined)  # Add dropout before the dense layers
combined = layers.Dense(128, activation="relu")(combined)

output = layers.Dense(2, activation="softmax")(combined)

print(
    "Num GPUs Available: ", len(tf.config.list_physical_devices("GPU"))
)  # check if GPU is available
# check which gpu is running
print("The current used GPU is: ", tf.test.gpu_device_name())

early_stopping = callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = optimizers.Adam(learning_rate=lr_schedule)


model = models.Model(inputs=[image_input, sensor_input], outputs=output)


model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

C:\Users\obada\AppData\Local\Temp\ipykernel_13032\2874080781.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=image_input)


Num GPUs Available:  0
The current used GPU is:  


In [ ]:
# k fold cross validation
k = 5
num_val_samples = len(X_img_train) // k
num_epochs = 50

In [205]:
class_weight = {0: 1.0, 1: 2.0}

# x trainng 
# y prediction

history = model.fit(
    [X_img_train, X_sensor_train],
    y_train,
    validation_data=([X_img_val, X_sensor_val], y_val),
    class_weight=class_weight,
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 1597s 4s/step - accuracy: 0.9552 - loss: 0.1146 - val_accuracy: 0.7301 - val_loss: 3.2916
Epoch 2/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 800s 2s/step - accuracy: 0.9918 - loss: 0.0265 - val_accuracy: 0.4141 - val_loss: 13.3735
Epoch 3/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 755s 2s/step - accuracy: 0.9929 - loss: 0.0224 - val_accuracy: 0.7289 - val_loss: 3.3014
Epoch 4/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 734s 2s/step - accuracy: 0.9990 - loss: 0.0047 - val_accuracy: 0.8319 - val_loss: 0.7892
Epoch 5/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 721s 2s/step - accuracy: 0.9962 - loss: 0.0139 - val_accuracy: 0.9042 - val_loss: 0.9630
Epoch 6/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 757s 2s/step - accuracy: 0.9999 - loss: 5.0656e-04 - val_accuracy: 0.8778 - val_loss: 1.3075
Epoch 7/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 770s 2s/step - accuracy: 0.9958 - loss: 0.0120 - val_accuracy: 0.9259 - val_loss: 0.6358
Epoch 8/20
349/349 ━━━━━━━━━━━━━━━━━━━━ 762s 2s/step - accuracy: 0.9987 - loss: 0.0052 - va

In [214]:
test_loss, test_accuracy = model.evaluate(
    [X_img_val, X_sensor_val],
    y_val,
    batch_size=32,
    verbose=1,
)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")

50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 389ms/step - accuracy: 0.6370 - loss: 0.6378
Validation Accuracy: 63.86%


In [3]:
# load the model 
import tensorflow as tf
from keras import models
model = models.load_model("multi_modal_posture_model.h5")

In [1]:
y_pred = model.predict([X_img_test, X_sensor_test], batch_size=32, verbose=1)

y_pred = y_pred.flatten()
y_test = y_test.flatten()

NameError: name 'model' is not defined

In [22]:
from sklearn.metrics import r2_score, mean_squared_error


# Calculate R-squared and Mean Squared Error
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R-squared (R²): {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

R-squared (R²): 0.0953
Mean Squared Error (MSE): 0.2262


In [216]:
test_loss, test_accuracy

(0.6346042156219482, 0.6385920643806458)

In [213]:
model.save("multi_modal_posture_model.h5")
# save as tensorflow light model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open("multi_modal_posture_model.tflite", "wb") as f:
    f.write(tflite_model)

2025-01-09 17:26:38,041 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-01-09 17:26:39,850 - INFO - Function `function` contains input name(s) resource with unsupported characters which will be renamed to functional_12_1_dense_38_1_biasadd_readvariableop_resource in the SavedModel.
2025-01-09 17:26:40,091 - INFO - Function `function` contains input name(s) resource with unsupported characters which will be renamed to functional_12_1_dense_38_1_biasadd_readvariableop_resource in the SavedModel.


INFO:tensorflow:Assets written to: C:\Users\obada\AppData\Local\Temp\tmp4c62n6_9\assets


2025-01-09 17:26:45,563 - INFO - Assets written to: C:\Users\obada\AppData\Local\Temp\tmp4c62n6_9\assets


Saved artifact at 'C:\Users\obada\AppData\Local\Temp\tmp4c62n6_9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_1920'), TensorSpec(shape=(None, 6), dtype=tf.float32, name='keras_tensor_2075')]
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2247513000592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247513008512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247513010448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247513004992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247504420304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247512999536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247512999712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247513006224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2247512998128: TensorSpec(shape=(), dtype=tf.resource, na

In [ ]:
impored_model = models.load_model("multi_modal_posture_model.h5")

In [219]:
!jupyter nbconvert --to script "C:\Users\obada\Desktop\HoldWise Dataset\posture_classifier.ipynb"
!jupyter nbconvert --to html "C:\Users\obada\Desktop\HoldWise Dataset\posture_classifier.ipynb"

C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
[NbConvertApp] Converting notebook C:\Users\obada\Desktop\HoldWise Dataset\posture_classifier.ipynb to script
[NbConvertApp] Writing 19544 bytes to C:\Users\obada\Desktop\HoldWise Dataset\posture_classifier.py
C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a d